In [14]:
import sagemaker
import boto3
from sagemaker import get_execution_role
sess = sagemaker.Session()
role = get_execution_role()
container = sagemaker.amazon.amazon_estimator.get_image_uri('us-east-1', 
                                                            'xgboost', 
                                                            'latest')

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [17]:
s3_train_data = \
    's3://bucket-mukul/data/'
s3_output_location = \
    's3://bucket-mukul/output/xgboost/'

In [18]:
sagemaker_model = sagemaker.estimator.Estimator(container,
    role,
    train_instance_count=1,
    train_instance_type='ml.c4.4xlarge',
    train_volume_size=30,
    train_max_run=360000,
    input_mode='File',
    output_path=s3_output_location,
    sagemaker_session=sess)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [19]:
sagemaker_model.set_hyperparameters(objective='binary:logistic',
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.7,
    silent=0,
    num_round=50)

In [20]:
train_data = sagemaker.session.s3_input(s3_train_data, 
    distribution='FullyReplicated',
    content_type='text/csv', 
    s3_data_type='S3Prefix')

validation_data = sagemaker.session.s3_input(s3_train_data, 
    distribution='FullyReplicated',
    content_type='text/csv', 
    s3_data_type='S3Prefix')

data_channels = {'train': train_data, 
                 'validation': validation_data}

sagemaker_model.fit(inputs=data_channels, 
                    logs=True)

The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


2022-01-16 22:36:37 Starting - Starting the training job...
2022-01-16 22:37:01 Starting - Launching requested ML instancesProfilerReport-1642372597: InProgress
......
2022-01-16 22:38:03 Starting - Preparing the instances for training.........
2022-01-16 22:39:23 Downloading - Downloading input data......
2022-01-16 22:40:36 Training - Downloading the training image..Arguments: train
[2022-01-16:22:40:52:INFO] Running standalone xgboost training.
[2022-01-16:22:40:52:INFO] File size need to be processed in the node: 7359.36mb. Available memory size in the node: 22187.84mb
[2022-01-16:22:40:52:INFO] Determined delimiter of CSV input is ','
[22:40:52] S3DistributionType set as FullyReplicated

2022-01-16 22:41:02 Training - Training image download completed. Training in progress.[22:41:27] 9598049x100 matrix with 959804900 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2022-01-16:22:41:27:INFO] Determined delimiter of CSV input is ','
[22:41:27] S3Di

# Applying and evaluating the model

In [24]:
# transformer = sagemaker_model.transformer(instance_count=1, 
#                                           instance_type='ml.m4.2xlarge',
#                                           output_path=s3_output_location)
# transformer.transform(s3_train_data, 
#                       content_type='text/csv', 
#                       split_type='Line')
# transformer.wait()